In [1]:
import spacy
import pandas as pd
import gensim

In [2]:
nlp = spacy.load('en')

In [25]:
output = pd.read_pickle("../dataset/inception_output_layer")

In [26]:
output.head(5)

,img,output layer,scores
0,029599.jpg,"[8.56041e-06, 7.97617e-06, 1.79152e-05, 3.7247...","{'alp': 0.967034, 'valley, vale': 0.0136182, '..."
1,013369.jpg,"[2.58352e-05, 1.43098e-05, 3.03804e-05, 1.8103...","{'breakwater, groin, groyne, mole, bulwark, se..."
2,023620.jpg,"[2.48684e-05, 2.80843e-05, 1.79409e-05, 1.1612...","{'barn': 0.948197, 'boathouse': 0.010576, 'wor..."
3,004139.jpg,"[0.00025399, 0.000173421, 0.000117245, 0.00025...","{'pot, flowerpot': 0.346745, 'patio, terrace':..."
4,036728.jpg,"[0.000127009, 5.8184e-05, 0.000140877, 0.00025...","{'jean, blue jean, denim': 0.312845, 'cellular..."


In [27]:
scores = output.scores

In [28]:
scores.iloc[0]

{'alp': 0.96703446,
 'cliff, drop, drop-off': 0.00088864344,
 'marmot': 0.00077028183,
 'mountain tent': 0.0034724902,
 'valley, vale': 0.013618188}

In [11]:
def convert_scores(score_key):
    image_scores = {}
    for text, score in score_key.items():
        doc = nlp(text)
        for token in doc:
            if(token.pos_ == "NOUN"):
                image_scores[token.lemma_] = score 
    return image_scores

In [34]:
output.drop(columns=["scores"], inplace=True)

In [42]:
cleaned = pd.Series(scores.apply(convert_scores))


In [37]:
cleaned.head()

0    {'alp': 0.967034, 'valley': 0.0136182, 'mounta...
1    {'groin': 0.952986, 'groyne': 0.952986, 'mole'...
2    {'barn': 0.948197, 'boathouse': 0.010576, 'wor...
3    {'pot': 0.346745, 'flowerpot': 0.346745, 'terr...
4    {'jean': 0.312845, 'denim': 0.312845, 'telepho...
Name: scores, dtype: object

In [38]:
output["scores"] = cleaned
output.head()

,img,output layer,scores
0,029599.jpg,"[8.56041e-06, 7.97617e-06, 1.79152e-05, 3.7247...","{'alp': 0.967034, 'valley': 0.0136182, 'mounta..."
1,013369.jpg,"[2.58352e-05, 1.43098e-05, 3.03804e-05, 1.8103...","{'groin': 0.952986, 'groyne': 0.952986, 'mole'..."
2,023620.jpg,"[2.48684e-05, 2.80843e-05, 1.79409e-05, 1.1612...","{'barn': 0.948197, 'boathouse': 0.010576, 'wor..."
3,004139.jpg,"[0.00025399, 0.000173421, 0.000117245, 0.00025...","{'pot': 0.346745, 'flowerpot': 0.346745, 'terr..."
4,036728.jpg,"[0.000127009, 5.8184e-05, 0.000140877, 0.00025...","{'jean': 0.312845, 'denim': 0.312845, 'telepho..."


In [39]:
with open("../dataset/inception_output_layer_f2", 'wb') as f:
        pkl.dump(output, f, protocol=2)

In [40]:
processed_scores = pd.read_pickle("../dataset/inception_output_layer_f2");

In [41]:
processed_scores.head()

,img,output layer,scores
0,029599.jpg,"[8.56041e-06, 7.97617e-06, 1.79152e-05, 3.7247...","{'alp': 0.967034, 'valley': 0.0136182, 'mounta..."
1,013369.jpg,"[2.58352e-05, 1.43098e-05, 3.03804e-05, 1.8103...","{'groin': 0.952986, 'groyne': 0.952986, 'mole'..."
2,023620.jpg,"[2.48684e-05, 2.80843e-05, 1.79409e-05, 1.1612...","{'barn': 0.948197, 'boathouse': 0.010576, 'wor..."
3,004139.jpg,"[0.00025399, 0.000173421, 0.000117245, 0.00025...","{'pot': 0.346745, 'flowerpot': 0.346745, 'terr..."
4,036728.jpg,"[0.000127009, 5.8184e-05, 0.000140877, 0.00025...","{'jean': 0.312845, 'denim': 0.312845, 'telepho..."


In [44]:
vocab = set()
processed_scores.scores.apply(lambda d: vocab.update(list(d.keys())))
pd.DataFrame({"vocab":list(vocab)}).to_csv("../dataset/vocab.csv")

In [52]:
new_model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [50]:
v_token = []
for t in list(vocab):
    if t  in new_model.vocab:
        v_token.append(t)
pd.DataFrame({"word2vec":v_token}).to_csv("../dataset/word2vecvocab.csv")

In [53]:
new_model.distances('cat',list(v_token))

/Users/aadilrasheed/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:759: RuntimeWarning: divide by zero encountered in true_divide
  similarities = dot_products / (norm * all_norms)


array([-inf, -inf, -inf, ..., -inf, -inf, -inf], dtype=float32)

In [55]:
new_model.most_similar_to_given('dog',list(v_token))

'dog'

In [57]:
new_model.most_similar('dog',topn=2)

[('dogs', 0.868048906326294), ('puppy', 0.8106427788734436)]

In [40]:
a=pd.read_csv("../dataset/word2vecvocab.csv")

In [41]:
b = pd.DataFrame( index= a["word2vec"], columns = ["image"])

In [43]:
b.head()

,image
word2vec,
dock,NaN
wagon,NaN
bullet,NaN
carton,NaN
balance,NaN


In [44]:
b.loc["dock"]["image"]

nan